In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchtext.transforms import SentencePieceTokenizer
from torch.hub import load_state_dict_from_url

### Data Load

In [ ]:
DATA_PATH = "./data/data.txt"
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(1337)

In [ ]:
# Transformer base hyperparams
block_size = 256
batch_size = 64
max_iters = 8000
eval_interval = 1000
learning_rate = 1e-4

# Model Size
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6

# Regularization || Optimization
dropout = 0.2
weight_decay = 1e-6

In [ ]:
vocab_size = len(chars)

# create a mapping from characters to integers (Replace with tiktoken) https://github.com/openai/tiktoken
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [
    stoi[c] for c in s
]  # encoder: take a string, output a list of integers
decode = lambda l: "".join(
    [itos[i] for i in l]
)  # decoder: take a list of integers, output a string

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)

n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

### Data batch

In [ ]:
def get_batch(split):
    """
    Generates a training or validation batch.

    Parameters:
    - split (str): Either "train" or "val" indicating the data split.

    Returns:
    - x (torch.Tensor): Input tensor of shape (batch_size, block_size).
    - y (torch.Tensor): Target tensor of shape (batch_size, block_size).
    """
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    """
    Estimates the mean loss on the training and validation sets.

    Returns:
    - out (dict): Dictionary containing mean losses for "train" and "val" splits.
    """
    
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Head of self-attention

In [ ]:
class Head(nn.Module):
    """
    One head of self-attention.

    Parameters:
    - head_size (int): The size of the attention head.
    """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Forward pass of the self-attention head.

        Parameters:
        - x (torch.Tensor): Input tensor of shape (B, T, C), where B is the batch size,
          T is the sequence length, and C is the number of channels.

        Returns:
        - out (torch.Tensor): Output tensor after self-attention of shape (B, T, head_size).
        """
        
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        # compute attention scores ("affinities")
        wei = (
            q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        ) 
        
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        
        v = self.value(x)
        out = wei @ v 
        return out

### MultiHeadAttention


In [ ]:
class MultiHeadAttention(nn.Module):
    """
    Multiple heads of self-attention in parallel.

    Parameters:
    - num_heads (int): The number of attention heads.
    - head_size (int): The size of each attention head.
    """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Forward pass of the multi-head self-attention.

        Parameters:
        - x (torch.Tensor): Input tensor of shape (B, T, C), where B is the batch size,
          T is the sequence length, and C is the number of channels.

        Returns:
        - out (torch.Tensor): Output tensor after multi-head self-attention of shape (B, T, n_embd).
        """
        
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

### FeedFoward Layer

In [ ]:
class FeedFoward(nn.Module):
    """
    A simple linear layer followed by a non-linearity and dropout.

    Parameters:
    - n_embd (int): The embedding size.

    Notes:
    The feedforward network consists of two linear layers with a SiLU activation
    function in between, followed by a dropout layer.

    Input:
    - x (torch.Tensor): Input tensor of shape (B, T, n_embd), where B is the batch size,
      T is the sequence length, and n_embd is the embedding size.

    Output:
    - out (torch.Tensor): Output tensor after the feedforward network of shape (B, T, n_embd).
    """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.SiLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        """
        Forward pass of the feedforward network.

        Parameters:
        - x (torch.Tensor): Input tensor of shape (B, T, n_embd), where B is the batch size,
          T is the sequence length, and n_embd is the embedding size.

        Returns:
        - out (torch.Tensor): Output tensor after the feedforward network of shape (B, T, n_embd).
        """
        return self.net(x)

### Transformer Block


In [ ]:
class Block(nn.Module):
    """
    Transformer Block: Communication followed by Computation.

    Parameters:
    - embedding_dim (int): The embedding dimension.
    - num_attention_heads (int): The number of attention heads.

    Notes:
    The block consists of a Multi-Head Self-Attention layer, followed by a Feedforward
    layer, both followed by Layer Normalization.

    Input:
    - x (torch.Tensor): Input tensor with shape (B, T, embedding_dim), where B is the batch size,
      T is the sequence length, and embedding_dim is the embedding dimension.

    Output:
    - out (torch.Tensor): Output tensor after the transformer block with shape (B, T, embedding_dim).
    """

    def __init__(self, n_embd, n_head):
        super().__init__()

        head_size = embedding_dim // num_attention_heads
        self.multi_head_attention = MultiHeadAttention(num_attention_heads, head_size)

        self.feedforward = FeedForward(embedding_dim)
        self.layer_norm1 = nn.LayerNorm(embedding_dim)
        self.layer_norm2 = nn.LayerNorm(embedding_dim)

    def forward(self, x):
        """
        Forward pass of the transformer block.

        Parameters:
        - x (torch.Tensor): Input tensor with shape (B, T, embedding_dim), where B is the batch size,
          T is the sequence length, and embedding_dim is the embedding dimension.

        Returns:
        - out (torch.Tensor): Output tensor after the transformer block with shape (B, T, embedding_dim).
        """

        x = x + self.multi_head_attention(self.layer_norm1(x))
        x = x + self.feedforward(self.layer_norm2(x))
        return x

### Generative Language Model using a Transformer architecture.

In [ ]:
class GenerativeLanguageModel(nn.Module):
    """
    Generative Language Model using a Transformer architecture.

    Attributes:
    - token_embedding_table (nn.Embedding): Lookup table for token embeddings.
    - position_embedding_table (nn.Embedding): Lookup table for position embeddings.
    - blocks (nn.Sequential): Transformer blocks for processing embeddings.
    - ln_f (nn.LayerNorm): Final layer normalization.
    - lm_head (nn.Linear): Linear layer for language modeling prediction.

    Methods:
    - forward(idx, targets=None): Forward pass of the model.
    - generate(idx, max_new_tokens): Generate new tokens based on the input indices.
    """

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        """
        The forward pass takes input indices, performs embedding lookup, adds positional embeddings,
        passes through transformer blocks, and produces language modeling predictions.
        
        Returns:
        - logits (torch.Tensor): Output logits from the language model.
        - loss (torch.Tensor): Language modeling loss, calculated only if targets are provided during training.
        """
        
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = pos_emb + tok_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)

            # Apply log_softmax to logits for better numerical stability
            log_probs = F.log_softmax(logits, dim=-1)

            # Use negative log-likelihood loss
            loss = F.nll_loss(log_probs, targets)

            # # cross_entropy
            # loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """
        The generation method extends the input sequence by sampling new tokens from the model's output.
        """
        
        for i in range(max_new_tokens):

            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            
            logits = logits[:, -1, :]
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            
            idx_next = torch.multinomial(probs, num_samples=1)

            idx = torch.cat((idx, idx_next), dim=1)
        return idx

### Create an instance of the GenerativeLanguageModel


In [ ]:
model = GenerativeLanguageModel()
m = model.to(device)

### Define the optimizer

In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(), lr=learning_rate, weight_decay=weight_decay
)

### Training loop

In [ ]:
for iter in range(max_iters):
    print(f"Epoch {iter+1}", end="\r")

    # Every once in a while, evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()

        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # Sample a batch of data
    xb, yb = get_batch("train")

    # Forward pass and compute the loss
    logits, loss = model(xb, yb)
    
    # Zero the gradients, perform backpropagation, and update the weights
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

Saving 

### Saving Model


In [ ]:
import pickle

model_pkl_file = "model_befoer_tuning.pkl"

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model, file)

torch.save(model.state_dict(), "model_befoer_tuning.pt")

with open(model_pkl_file, "wb") as file:
    pickle.dump(model, file)

### Loading Model

In [ ]:
model = GenerativeLanguageModel()
m = model.to(device)
model.load_state_dict(torch.load('model_befoer_tuning.pt'))

### Model Test 

In [ ]:
# Input example
input_text = "Hi Junior, i need some "

context = torch.tensor(encode(input_text), dtype=torch.long, device=device).unsqueeze(0)

# Evaluate model
y_predict = decode(m.generate(context, max_new_tokens=500)[0].tolist())

print(y_predict)